In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('./pruebillas.ipynb'), '..'))
os.chdir(root_dir)

sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas as pd
import pandas_ta as pandas_ta
import MetaTrader5 as mt5
import pandas as pd
from get_data import get_data
import pytz
from datetime import datetime
import talib as ta
import random
from backbone.trader_bot import TraderBot
random.seed(42)


In [3]:
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
 
# get all symbols
symbols = mt5.symbols_get()


groups = []
for symbol in symbols:
    group = symbol.path.split('\\')[0]
    if not group in groups:
        groups.append(group)
groups


['Forex_Indicator',
 'CryptoCross_grp',
 'Crypto_group',
 'Energies_group',
 'Forex_group',
 'Indices_group',
 'Stocks_group']

In [4]:

tickers = [symbol.path.split('\\')[1] for symbol in symbols if 'Energies_group' in symbol.path]# and symbol.spread < 20]
print(tickers)


# Establecer la zona horaria a UTC
timezone = pytz.timezone("Etc/UTC")

# Crear objetos 'datetime' en zona horaria UTC
date_from = datetime(2021, 10, 1, tzinfo=timezone)
date_to = datetime(2024, 9, 1, tzinfo=timezone)

intervals = [
    # mt5.TIMEFRAME_M1,
    # mt5.TIMEFRAME_M2,
    # mt5.TIMEFRAME_D1,
    # mt5.TIMEFRAME_M3,
    # mt5.TIMEFRAME_M4,
    # mt5.TIMEFRAME_M5,
    mt5.TIMEFRAME_M15,
    mt5.TIMEFRAME_M30,
    mt5.TIMEFRAME_H1,
    mt5.TIMEFRAME_M30,
]

symbols = get_data(tickers, intervals, date_from, date_to)

['UKOILm', 'USOILm', 'XNGUSDm']
MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288
UKOILm
UKOILm
UKOILm
UKOILm
USOILm
USOILm
USOILm
USOILm
XNGUSDm
XNGUSDm
XNGUSDm
XNGUSDm


In [5]:
max_start_date = None
intervals_start_dates = {}

for interval in intervals:
    for ticker in tickers:
        if not max_start_date or symbols[ticker][interval].index.min() > max_start_date:
            max_start_date = symbols[ticker][interval].index.min()
        
    intervals_start_dates[interval] = max_start_date

intervals_start_dates



{15: Timestamp('2022-03-20 22:15:00'),
 30: Timestamp('2022-03-20 22:15:00'),
 16385: Timestamp('2022-03-20 22:15:00')}

In [6]:
from pandas import Timestamp
limit_date_train = Timestamp('2022-10-01 00:00:00')
limit_date_train

Timestamp('2022-10-01 00:00:00')

In [7]:
def ll_hh_indicator(close, window=None):
    if type(close) != pd.Series:
        close = close.s

    rolling_min = close.rolling(window=window, min_periods=1).min()
    is_lower_low = close == rolling_min

    rolling_max = close.rolling(window=window, min_periods=1).max()
    is_higher_high = close == rolling_max

    return is_lower_low, is_higher_high


def trend_indicator(close, window):
    if type(close) != pd.Series:
        close = close.s
    
    shifted = close.shift(window)
    up_trend = close > shifted
    
    return up_trend


class Breakout(Strategy):

    trend_window = 100
    ll_hh_window = 50
    risk = 1
    
    bars_in_trade = 0
    
    def init(self):
        
        self.adx = self.I(
            ta.ADX, 
            self.data.High, 
            self.data.Low, 
            self.data.Close
        )
        
        self.ll, self.hh = self.I(
            ll_hh_indicator, 
            self.data.Close, 
            # self.data.High, 
            # self.data.Low, 
            window=self.ll_hh_window
        )
        
        self.up_trend = self.I(trend_indicator, self.data.Close, window=self.trend_window)

    def next(self):
        
        if self.adx < 35:
            return

        if self.position:
            self.bars_in_trade += 1
            
            if self.bars_in_trade >= 5:
                self.position.close()
                self.bars_in_trade = 0

            
            
            # if self.position.is_long:
            #     if not self.up_trend and self.ll:
            #         self.buy(size=self.risk / 100)

            # if self.position.is_short:
            #     if self.up_trend and self.hh:
            #         self.sell(size=self.risk / 100)

        else:
               
                
            if self.hh and self.up_trend:
                self.sell(size=self.risk / 100)
                
            if self.ll and not self.up_trend:
                self.buy(size=self.risk / 100)
                


In [8]:
import itertools
import numpy as np
from sklearn.linear_model import LinearRegression


strategies = [
    Breakout, 
]

experiments = parameter_combinations = list(itertools.product(
    tickers, intervals, strategies
))

performance = pd.DataFrame()

for ticker, interval, strategy in experiments:

    start_date = intervals_start_dates[interval]

    bt_train = Backtest(
        symbols[ticker][interval].loc[start_date:limit_date_train], 
        strategy,
        commission=7e-4,
        cash=15_000, 
        margin=1/30
    )
    
    stats = bt_train.run()
    
    equity_curve = stats._equity_curve['Equity'].values    
    x = np.arange(len(equity_curve)).reshape(-1, 1)
    reg = LinearRegression().fit(x, equity_curve)
    stability_ratio = reg.score(x, equity_curve)

    df_stats = pd.DataFrame({
        'strategy':[strategy.__name__],
        'ticker':[ticker],
        'interval':[interval],
        'stability_ratio':[stability_ratio],
        'return':[stats['Return [%]']],
        'final_eq':[stats['Equity Final [$]']],
        'drawdown':[stats['Max. Drawdown [%]']],
        'drawdown_duration':[stats['Max. Drawdown Duration']],
        'win_rate':[stats['Win Rate [%]']], 
        'sharpe_ratio':[stats['Sharpe Ratio']],
        'trades':[stats['# Trades']],
        'avg_trade_percent':[stats['Avg. Trade [%]']],
        'exposure':[stats['Exposure Time [%]']],
        'final_equity':[stats['Equity Final [$]']],
        'Duration':[stats['Duration']],
    })

    performance = pd.concat([performance, df_stats])

performance['return/dd'] = performance['return'] / -performance['drawdown']
performance['drawdown'] = -performance['drawdown']
performance['custom_metric'] = (performance['return'] / (1 + performance.drawdown)) * np.log(1 + performance.trades)
# performance.drawdown_duration = performance.drawdown_duration.dt.days



In [9]:
performance

,strategy,ticker,interval,stability_ratio,return,final_eq,drawdown,drawdown_duration,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd,custom_metric
0,Breakout,UKOILm,15,0.829078,11.603352,16740.502792,3.134106,47 days 00:00:00,52.054795,2.809669,146,0.247884,23.811574,16740.502792,193 days 20:45:00,3.702285,14.006838
0,Breakout,UKOILm,30,0.647978,6.419804,15962.970574,3.375692,63 days 06:00:00,49.152542,1.855174,59,0.344034,25.482094,15962.970574,193 days 20:30:00,1.901774,6.007024
0,Breakout,UKOILm,16385,0.518699,4.995958,15749.393647,3.986433,93 days 07:00:00,51.851852,1.326110,27,0.576744,19.683414,15749.393647,193 days 20:00:00,1.253240,3.338570
0,Breakout,UKOILm,30,0.647978,6.419804,15962.970574,3.375692,63 days 06:00:00,49.152542,1.855174,59,0.344034,25.482094,15962.970574,193 days 20:30:00,1.901774,6.007024
0,Breakout,USOILm,15,0.706734,-3.840289,14423.956581,6.497430,179 days 05:30:00,40.939597,0.000000,149,-0.092769,12.570476,14423.956581,193 days 22:15:00,-0.591047,-2.566518
0,Breakout,USOILm,30,0.530503,6.039159,15905.873787,2.341300,87 days 05:00:00,48.648649,1.521806,74,0.255797,17.847604,15905.873787,193 days 22:00:00,2.579404,7.803547
0,Breakout,USOILm,16385,0.912978,9.425872,16413.880816,2.352199,50 days 03:00:00,55.555556,2.266736,36,0.824689,27.167085,16413.880816,193 days 21:00:00,4.007260,10.153351
0,Breakout,USOILm,30,0.530503,6.039159,15905.873787,2.341300,87 days 05:00:00,48.648649,1.521806,74,0.255797,17.847604,15905.873787,193 days 22:00:00,2.579404,7.803547
0,Breakout,XNGUSDm,15,0.756986,-3.919016,14412.147597,9.203824,178 days 10:00:00,46.857143,0.000000,175,-0.083448,12.921483,14412.147597,193 days 22:15:00,-0.425803,-1.985845
0,Breakout,XNGUSDm,30,0.229685,-4.569813,14314.528117,10.901369,94 days 02:30:00,40.000000,0.000000,95,-0.180063,13.394120,14314.528117,193 days 22:00:00,-0.419196,-1.752590


In [10]:

filter_performance = performance[
    (performance['return']>0) 
    # & (performance['return/dd']>=2) 
    # & (performance['stability_ratio'] > 0.8)
    # & (performance['interval'] <= 16385)
].sort_values(by=['custom_metric'], ascending=[False])#.drop_duplicates(subset=['ticker'], keep='first')

portfolio = filter_performance.ticker.tolist()
intervals = filter_performance.interval.values.tolist()

display(filter_performance)

# portfolio

,strategy,ticker,interval,stability_ratio,return,final_eq,drawdown,drawdown_duration,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd,custom_metric
0,Breakout,UKOILm,15,0.829078,11.603352,16740.502792,3.134106,47 days 00:00:00,52.054795,2.809669,146,0.247884,23.811574,16740.502792,193 days 20:45:00,3.702285,14.006838
0,Breakout,USOILm,16385,0.912978,9.425872,16413.880816,2.352199,50 days 03:00:00,55.555556,2.266736,36,0.824689,27.167085,16413.880816,193 days 21:00:00,4.007260,10.153351
0,Breakout,USOILm,30,0.530503,6.039159,15905.873787,2.341300,87 days 05:00:00,48.648649,1.521806,74,0.255797,17.847604,15905.873787,193 days 22:00:00,2.579404,7.803547
0,Breakout,USOILm,30,0.530503,6.039159,15905.873787,2.341300,87 days 05:00:00,48.648649,1.521806,74,0.255797,17.847604,15905.873787,193 days 22:00:00,2.579404,7.803547
0,Breakout,UKOILm,30,0.647978,6.419804,15962.970574,3.375692,63 days 06:00:00,49.152542,1.855174,59,0.344034,25.482094,15962.970574,193 days 20:30:00,1.901774,6.007024
0,Breakout,UKOILm,30,0.647978,6.419804,15962.970574,3.375692,63 days 06:00:00,49.152542,1.855174,59,0.344034,25.482094,15962.970574,193 days 20:30:00,1.901774,6.007024
0,Breakout,UKOILm,16385,0.518699,4.995958,15749.393647,3.986433,93 days 07:00:00,51.851852,1.326110,27,0.576744,19.683414,15749.393647,193 days 20:00:00,1.253240,3.338570


In [11]:
for ticker, interval in zip(portfolio, intervals):
    bt_train = Backtest(
        symbols[ticker][interval], 
        Breakout,
        commission=7e-4,
        cash=15_000, 
        margin=1/30
    )

    stats = bt_train.run()

    bt_train.plot(filename=f'./{ticker}.html', resample=False)

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarn

In [13]:
from utils import walk_forward
from utils import plot_full_equity_curve
from utils import get_wfo_stats

def  optim_func(series):
    return (series['Return [%]'] /  (1 + (-1*series['Max. Drawdown [%]']))) * np.log(1 + series['# Trades'])

def optim_func_2(stats):
    equity_curve = stats._equity_curve['Equity'].values    
    x = np.arange(len(equity_curve)).reshape(-1, 1)
    reg = LinearRegression().fit(x, equity_curve)
    stability_ratio = reg.score(x, equity_curve)
    return stability_ratio


for ticker, interval in zip(portfolio, intervals):
    
    print(ticker, interval)
    
    lookback_bars = 1000
    validation_bars = 250
    warmup_bars = 400

    params = {
        'cum_rsi_open_threshold' : [55, 65, 75, 85, 90],
        'cum_rsi_close_threshold' : [55, 65, 75, 85, 90],
        'maximize': optim_func_2
    }

    stats = walk_forward(
        Macd,
        symbols[ticker][interval], 
        lookback_bars=lookback_bars,
        validation_bars=validation_bars,
        warmup_bars=warmup_bars, 
        params=params,
        commission=7e-4, 
        margin=1/30, 
        cash=15_000
    )

    plot_full_equity_curve(
        symbols[ticker][interval], 
        stats, 
        warmup_bars=warmup_bars,
        lookback_bars=lookback_bars, 
        overlay_price=True
    )

    wfo_stats = get_wfo_stats(stats, warmup_bars, symbols[ticker][interval])

    for k, v in wfo_stats.items():
        print(k, v)